In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/ResNet18")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]

#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    train_loss_TL = []
    train_accuracy_TL = []
    test1_accuracy_TL = []
    test3_accuracy_TL = []
    test5_accuracy_TL = []
    #建立模型
    model = epm.ResNet_18(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("TL: ", TL+1, "fold: ", fold)
        print("----------------------------------------------------------------------------------------------------------------")
        #資料抓入
        train, test_loader1, test_loader3, test_loader5 = epg.getTrainDataset_val135(fold, sound[TL], num_class)

        #分批方法
        train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
        
        #優化器
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        #調度器
        scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        #訓練模型
        train_loss, train_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[TL], num_epochs, train_loader, test_loader1, test_loader3, test_loader5)
    
        #將gpu資料取回
        train_loss = torch.stack(train_loss,dim=-1).cpu()
        train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
    
        for i in range(num_epochs):
            # 寫入資料
            f.write("TL :{} fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(TL+1 ,fold, i+1, num_epochs, train_loss[i], train_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        train_loss_TL.append(train_loss)
        train_accuracy_TL.append(train_accuracy)
        test1_accuracy_TL.append(test1_accuracy)
        test3_accuracy_TL.append(test3_accuracy)
        test5_accuracy_TL.append(test5_accuracy)
    modelName = "Fold_" + str(fold) + "_TF.pkl"
    torch.save(model, modelName)
    train_loss_array.append(train_loss_TL)
    train_accuracy_array.append(train_accuracy_TL)
    test1_accuracy_array.append(test1_accuracy_TL)
    test3_accuracy_array.append(test3_accuracy_TL)
    test5_accuracy_array.append(test5_accuracy_TL)
f.close()

----------------------------------------------------------------------------------------------------------------
TL:  1 fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 0.6448878645896912 Train_acc: 55.375000%
Test accuracy of the model: 44.500000%
Test accuracy of the model: 36.250000%
Test accuracy of the model: 32.500000%
Train Epoch: 2/30 Train_Loss: 0.5061922073364258 Train_acc: 78.375000%
Test accuracy of the model: 65.500000%
Test accuracy of the model: 71.666667%
Test accuracy of the model: 68.750000%
Train Epoch: 3/30 Train_Loss: 0.30898216366767883 Train_acc: 85.062500%
Test accuracy of the model: 65.000000%
Test accuracy of the model: 64.583333%
Test accuracy of the model: 63.750000%
Train Epoch: 4/30 Train_Loss: 0.5386292934417725 Train_acc: 88.437500%
Test accuracy of the model: 61.500000%
Test accuracy of the model: 66.666667%
Test accuracy of the model: 65.000000%
Train

Test accuracy of the model: 77.500000%
Train Epoch: 12/30 Train_Loss: 0.11814169585704803 Train_acc: 99.062508%
Test accuracy of the model: 67.750000%
Test accuracy of the model: 75.833333%
Test accuracy of the model: 81.250000%
Train Epoch: 13/30 Train_Loss: 0.03565501049160957 Train_acc: 99.062508%
Test accuracy of the model: 65.750000%
Test accuracy of the model: 74.583333%
Test accuracy of the model: 80.000000%
Train Epoch: 14/30 Train_Loss: 0.006934488657861948 Train_acc: 99.062508%
Test accuracy of the model: 66.750000%
Test accuracy of the model: 76.250000%
Test accuracy of the model: 80.000000%
Train Epoch: 15/30 Train_Loss: 0.033127397298812866 Train_acc: 99.062508%
Test accuracy of the model: 66.000000%
Test accuracy of the model: 75.416667%
Test accuracy of the model: 80.000000%
Train Epoch: 16/30 Train_Loss: 0.061839278787374496 Train_acc: 99.062508%
Test accuracy of the model: 67.000000%
Test accuracy of the model: 75.416667%
Test accuracy of the model: 80.000000%
Train Ep

Test accuracy of the model: 69.250000%
Test accuracy of the model: 77.083333%
Test accuracy of the model: 81.250000%
Train Epoch: 24/30 Train_Loss: 0.0009445180767215788 Train_acc: 100.000000%
Test accuracy of the model: 68.500000%
Test accuracy of the model: 76.666667%
Test accuracy of the model: 81.250000%
Train Epoch: 25/30 Train_Loss: 0.0006840886198915541 Train_acc: 100.000000%
Test accuracy of the model: 69.000000%
Test accuracy of the model: 77.083333%
Test accuracy of the model: 81.250000%
Train Epoch: 26/30 Train_Loss: 0.00038365976070053875 Train_acc: 100.000000%
Test accuracy of the model: 69.250000%
Test accuracy of the model: 76.666667%
Test accuracy of the model: 81.250000%
Train Epoch: 27/30 Train_Loss: 0.00028490382828749716 Train_acc: 100.000000%
Test accuracy of the model: 69.250000%
Test accuracy of the model: 76.666667%
Test accuracy of the model: 81.250000%
Train Epoch: 28/30 Train_Loss: 0.00023000306100584567 Train_acc: 100.000000%
Test accuracy of the model: 69.5

Train Epoch: 4/30 Train_Loss: 0.03721238300204277 Train_acc: 98.020836%
Test accuracy of the model: 69.500000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 82.500000%
Train Epoch: 5/30 Train_Loss: 0.06138855591416359 Train_acc: 97.395836%
Test accuracy of the model: 76.000000%
Test accuracy of the model: 89.583333%
Test accuracy of the model: 95.000000%
Train Epoch: 6/30 Train_Loss: 0.05261087790131569 Train_acc: 97.604172%
Test accuracy of the model: 65.250000%
Test accuracy of the model: 77.916667%
Test accuracy of the model: 81.250000%
Train Epoch: 7/30 Train_Loss: 0.052322596311569214 Train_acc: 97.708336%
Test accuracy of the model: 72.750000%
Test accuracy of the model: 84.166667%
Test accuracy of the model: 88.750000%
Train Epoch: 8/30 Train_Loss: 0.05890901759266853 Train_acc: 98.333336%
Test accuracy of the model: 79.500000%
Test accuracy of the model: 90.416667%
Test accuracy of the model: 95.000000%
Train Epoch: 9/30 Train_Loss: 0.061007190495729446 Tra

Train Epoch: 16/30 Train_Loss: 0.004659054335206747 Train_acc: 100.000000%
Test accuracy of the model: 72.000000%
Test accuracy of the model: 85.000000%
Test accuracy of the model: 87.500000%
Train Epoch: 17/30 Train_Loss: 0.0019616608042269945 Train_acc: 100.000000%
Test accuracy of the model: 72.000000%
Test accuracy of the model: 82.916667%
Test accuracy of the model: 87.500000%
Train Epoch: 18/30 Train_Loss: 0.0012106499634683132 Train_acc: 99.687500%
Test accuracy of the model: 72.750000%
Test accuracy of the model: 85.416667%
Test accuracy of the model: 87.500000%
Train Epoch: 19/30 Train_Loss: 0.0016671064076945186 Train_acc: 100.000000%
Test accuracy of the model: 73.250000%
Test accuracy of the model: 85.833333%
Test accuracy of the model: 86.250000%
Train Epoch: 20/30 Train_Loss: 0.009715229272842407 Train_acc: 100.000000%
Test accuracy of the model: 73.250000%
Test accuracy of the model: 86.666667%
Test accuracy of the model: 88.750000%
Train Epoch: 21/30 Train_Loss: 0.00179

...

Test accuracy of the model: 73.750000%
Test accuracy of the model: 80.000000%
Test accuracy of the model: 83.750000%
Train Epoch: 9/30 Train_Loss: 0.0032449671998620033 Train_acc: 100.000000%
Test accuracy of the model: 69.750000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 86.250000%
Train Epoch: 10/30 Train_Loss: 0.0009689947473816574 Train_acc: 100.000000%
Test accuracy of the model: 69.250000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 86.250000%
Train Epoch: 11/30 Train_Loss: 0.003246907377615571 Train_acc: 100.000000%
Test accuracy of the model: 69.000000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 86.250000%
Train Epoch: 12/30 Train_Loss: 0.0033469651825726032 Train_acc: 100.000000%
Test accuracy of the model: 69.250000%
Test accuracy of the model: 81.250000%
Test accuracy of the model: 86.250000%
Train Epoch: 13/30 Train_Loss: 0.0011109652696177363 Train_acc: 100.000000%
Test accuracy of the model: 69.250000

Train Epoch: 20/30 Train_Loss: 0.08360842615365982 Train_acc: 94.875000%
Test accuracy of the model: 76.500000%
Test accuracy of the model: 81.250000%
Test accuracy of the model: 76.250000%
Train Epoch: 21/30 Train_Loss: 0.08292084187269211 Train_acc: 94.562500%
Test accuracy of the model: 77.500000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 82.500000%
Train Epoch: 22/30 Train_Loss: 0.12780000269412994 Train_acc: 94.750000%
Test accuracy of the model: 78.000000%
Test accuracy of the model: 79.166667%
Test accuracy of the model: 80.000000%
Train Epoch: 23/30 Train_Loss: 0.0399533212184906 Train_acc: 94.312500%
Test accuracy of the model: 76.750000%
Test accuracy of the model: 75.416667%
Test accuracy of the model: 80.000000%
Train Epoch: 24/30 Train_Loss: 0.09439604729413986 Train_acc: 94.937500%
Test accuracy of the model: 79.250000%
Test accuracy of the model: 80.833333%
Test accuracy of the model: 81.250000%
Train Epoch: 25/30 Train_Loss: 0.14550140500068665 

Train Epoch: 1/30 Train_Loss: 0.0027488465420901775 Train_acc: 99.687500%
Test accuracy of the model: 61.250000%
Test accuracy of the model: 68.750000%
Test accuracy of the model: 72.500000%
Train Epoch: 2/30 Train_Loss: 0.01033018622547388 Train_acc: 99.687500%
Test accuracy of the model: 58.750000%
Test accuracy of the model: 68.333333%
Test accuracy of the model: 75.000000%
Train Epoch: 3/30 Train_Loss: 0.02343897894024849 Train_acc: 99.687500%
Test accuracy of the model: 65.250000%
Test accuracy of the model: 74.583333%
Test accuracy of the model: 80.000000%
Train Epoch: 4/30 Train_Loss: 0.002194416942074895 Train_acc: 100.000000%
Test accuracy of the model: 67.000000%
Test accuracy of the model: 75.833333%
Test accuracy of the model: 81.250000%
Train Epoch: 5/30 Train_Loss: 0.0033089728094637394 Train_acc: 100.000000%
Test accuracy of the model: 68.500000%
Test accuracy of the model: 77.916667%
Test accuracy of the model: 83.750000%
Train Epoch: 6/30 Train_Loss: 0.0288595575839281

Test accuracy of the model: 81.250000%
Train Epoch: 13/30 Train_Loss: 0.06611555069684982 Train_acc: 93.687500%
Test accuracy of the model: 76.750000%
Test accuracy of the model: 81.666667%
Test accuracy of the model: 78.750000%
Train Epoch: 14/30 Train_Loss: 0.19624057412147522 Train_acc: 94.000000%
Test accuracy of the model: 78.000000%
Test accuracy of the model: 79.166667%
Test accuracy of the model: 78.750000%
Train Epoch: 15/30 Train_Loss: 0.183035746216774 Train_acc: 93.687500%
Test accuracy of the model: 78.500000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 77.500000%
Train Epoch: 16/30 Train_Loss: 0.1052074059844017 Train_acc: 93.937500%
Test accuracy of the model: 75.000000%
Test accuracy of the model: 77.083333%
Test accuracy of the model: 75.000000%
Train Epoch: 17/30 Train_Loss: 0.23244376480579376 Train_acc: 93.812500%
Test accuracy of the model: 76.500000%
Test accuracy of the model: 81.250000%
Test accuracy of the model: 77.500000%
Train Epoch: 1

Test accuracy of the model: 86.250000%
Train Epoch: 25/30 Train_Loss: 0.11510433256626129 Train_acc: 98.645836%
Test accuracy of the model: 75.750000%
Test accuracy of the model: 88.333333%
Test accuracy of the model: 88.750000%
Train Epoch: 26/30 Train_Loss: 0.0596887469291687 Train_acc: 98.645836%
Test accuracy of the model: 74.750000%
Test accuracy of the model: 87.916667%
Test accuracy of the model: 87.500000%
Train Epoch: 27/30 Train_Loss: 0.07644828408956528 Train_acc: 98.645836%
Test accuracy of the model: 75.500000%
Test accuracy of the model: 87.916667%
Test accuracy of the model: 87.500000%
Train Epoch: 28/30 Train_Loss: 0.0693536326289177 Train_acc: 98.645836%
Test accuracy of the model: 75.500000%
Test accuracy of the model: 87.500000%
Test accuracy of the model: 88.750000%
Train Epoch: 29/30 Train_Loss: 0.025716356933116913 Train_acc: 98.645836%
Test accuracy of the model: 74.500000%
Test accuracy of the model: 86.666667%
Test accuracy of the model: 87.500000%
Train Epoch:

In [4]:
#設定存檔位置
f = open("lastData.txt", "w")
for TL in range(3):
    train_loss_last = []
    train_accuracy_last = []
    test1_accuracy_last = []
    test3_accuracy_last = []
    test5_accuracy_last = []
    
    #整理出訓練完的數值
    for fold in range(5):
        train_loss_last.append(train_loss_array[fold][TL][num_epochs-1])
        train_accuracy_last.append(train_accuracy_array[fold][TL][num_epochs-1])
        test1_accuracy_last.append(test1_accuracy_array[fold][TL][num_epochs-1])
        test3_accuracy_last.append(test3_accuracy_array[fold][TL][num_epochs-1])
        test5_accuracy_last.append(test5_accuracy_array[fold][TL][num_epochs-1])
        # 寫入資料
        f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][TL][num_epochs-1], train_accuracy_array[fold][TL][num_epochs-1], test1_accuracy_array[fold][TL][num_epochs-1], test3_accuracy_array[fold][TL][num_epochs-1], test5_accuracy_array[fold][TL][num_epochs-1]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    # 寫入資料
    f.write("Max Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
    f.write("Min Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
    f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
f.close()